# Khám phá dữ liệu

Tạm thời dừng việc làm sạch dữ liệu, ta chuyển sang xem có thể rút ra được những nhận xét gì từ bộ data này.

Trước hết cứ load lại dữ liệu

In [12]:
# %load loaddata.py
import pandas as pd

pd.set_option('max.rows', 14)

data = pd.read_csv(
    'data/titanic.csv',
    index_col='PassengerId',
    usecols=lambda col: col not in ('Cabin', 'Ticket'),
    dtype={
        'Pclass': 'category',
        'Sex': 'category',
        'Embarked': 'category'
    }
)

data['Age'] = (data['Age'].fillna(data['Age'].mean())
                          .astype(int))
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null category
Name        891 non-null object
Sex         891 non-null category
Age         891 non-null int64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null category
dtypes: category(3), float64(1), int64(4), object(1)
memory usage: 51.6+ KB


Ngoài `info()` thì còn có hàm `describe()` mô tả cụ thể hơn

In [14]:
data.describe()

,Survived,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,29.544332,0.523008,0.381594,32.204208
std,0.486592,13.013778,1.102743,0.806057,49.693429
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,29.000000,0.000000,0.000000,14.454200
75%,1.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,80.000000,8.000000,6.000000,512.329200


Mặc định thì `describe()` chỉ mô tả các cột có kiểu dữ liệu số. Nếu muốn hiện hết thì có thể dùng parameter `include`

In [15]:
data.describe(include='all')

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891,891,891,891.000000,891.000000,891.000000,891.000000,891
unique,NaN,3,891,2,NaN,NaN,NaN,NaN,3
top,NaN,3,"Leeni, Mr. Fahim (""Philip Zenni"")",male,NaN,NaN,NaN,NaN,S
freq,NaN,491,1,577,NaN,NaN,NaN,NaN,646
mean,0.383838,NaN,NaN,NaN,29.544332,0.523008,0.381594,32.204208,NaN
std,0.486592,NaN,NaN,NaN,13.013778,1.102743,0.806057,49.693429,NaN
min,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,NaN,NaN,22.000000,0.000000,0.000000,7.910400,NaN
50%,0.000000,NaN,NaN,NaN,29.000000,0.000000,0.000000,14.454200,NaN
75%,1.000000,NaN,NaN,NaN,35.000000,1.000000,0.000000,31.000000,NaN


Cũng có thể dùng `include` để chọn ra những loại dữ liệu cụ thể

In [17]:
import numpy as np

data.describe(include=[np.number, 'category'])

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891,891,891.000000,891.000000,891.000000,891.000000,891
unique,NaN,3,2,NaN,NaN,NaN,NaN,3
top,NaN,3,male,NaN,NaN,NaN,NaN,S
freq,NaN,491,577,NaN,NaN,NaN,NaN,646
mean,0.383838,NaN,NaN,29.544332,0.523008,0.381594,32.204208,NaN
std,0.486592,NaN,NaN,13.013778,1.102743,0.806057,49.693429,NaN
min,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,NaN,22.000000,0.000000,0.000000,7.910400,NaN
50%,0.000000,NaN,NaN,29.000000,0.000000,0.000000,14.454200,NaN
75%,1.000000,NaN,NaN,35.000000,1.000000,0.000000,31.000000,NaN


### Exercise

Dùng `describe()` để mô tả riêng cột `Embarked`

In [18]:
# Code vào đây

In [20]:
# %load solutions/describe_embarked.py
data['Embarked'].describe()

count     891
unique      3
top         S
freq      646
Name: Embarked, dtype: object